In [85]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel #tensorflow GPT2 LM
import numpy as np
import torch
from tqdm import tqdm  # 진행률 표시를 위한 tqdm 라이브러리
from scipy.stats import norm #p-value 를 위함


mirrored_strategy = tf.distribute.MirroredStrategy()
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


KeyboardInterrupt: 

In [ ]:
filename = 'prompt.txt'
f = open(filename, 'r')
lines = f.readlines()
print(lines)
print('-' * 10)

['아침 일과 시작하기\n', '책 읽기의 즐거움\n', '가족과 함께 하는 저녁 식사\n', '친구와의 영화 관람\n', '주말 동안의 여행 계획\n', '정원 가꾸기의 효과\n', '요리를 통한 스트레스 해소\n', '자전거 타기의 장점\n', '집안 정리 정돈 팁\n', '개인 운동 루틴 유지하기\n', '취미 활동의 중요성\n', '새로운 언어 배우기\n', '음악 연주와 그 영향\n', '사진 촬영을 통한 추억 만들기\n', '지역 사회 봉사의 보람\n', '패션과 개인 스타일 표현\n', '휴일을 활용한 가족 모임\n', '건강한 식생활을 위한 조리법\n', '애완동물과 보내는 시간\n', '독서 클럽에서의 토론\n', '도서관 방문의 장점\n', '집에서의 필름 감상\n', '커피 한 잔과의 여유\n', '일기 쓰기의 치유 효과\n', '홈 베이킹의 즐거움\n', '직접 만든 공예품\n', '새로운 요리법 시도하기\n', '지역 공연 관람\n', '자연 속에서의 하이킹\n', '가구 리폼 프로젝트\n', '아이와 함께하는 공원 나들이\n', '새로운 스포츠 도전\n', '개인 기록의 개선\n', '집에서의 요가 세션\n', '반려동물과의 산책\n', '가족 앨범 만들기\n', '매일의 명상 시간\n', '가벼운 아침 조깅\n', '주말 브런치 준비\n', '일상의 소소한 행복 찾기\n', '스마트폰 없는 하루\n', '책상 정리의 중요성\n', '옷장 정리하기\n', '새벽 시간의 고요함\n', '오후의 티타임\n', '자기 전 명상\n', '집안 식물 가꾸기\n', '근처 박물관 탐방\n', '집에서의 칵테일 파티\n', '커뮤니티 클래스 참여\n', '셀프 케어 루틴\n', '장거리 걷기의 이점\n', '텃밭 가꾸기\n', '개인적인 스타일 개발\n', '책 추천 공유\n', '로컬 마켓 쇼핑\n', '장기 게임의 즐거움\n', '가족과의 보드게임 밤\n', '디지털 디톡스\n', '가까운 도시 탐험\n', '수제 비누 만들기\n', '로컬 아

In [120]:
def detection(final_sequence, logit_shape, gamma, initial_ids) :
    
    input_ids = tokenizer.encode(final_sequence, return_tensors='tf')
    input_ids_list = input_ids.numpy().tolist()[0]
    prev_tk = input_ids[0]
    green_counts = 0
    
    seed = initial_ids[-1]
    np.random.seed(seed)
    indices = np.arange(logit_shape) #0~51999
    green_list_size = int(logit_shape * gamma)  #25600
    green_list = np.random.choice(indices, green_list_size, replace=False)
    
    num = 0
    for id in input_ids_list :
        
        if id in green_list :
            green_counts = green_counts + 1
        
        seed = id % logit_shape
        #print(f"seed in detection : {seed}")
        np.random.seed(seed)
        
        indices = np.arange(logit_shape) #0~51999
        green_list_size = int(logit_shape * gamma)  #25600
        green_list = np.random.choice(indices, green_list_size, replace=False)
       #print(f"detection greenList : {green_list}")
       
        num += 1
        

    print(f"green list 단어 개수 : {green_counts}")
    print(f"red list 단어 개수 : {len(input_ids_list)-green_counts}")
    green_proportion = green_counts/len(input_ids_list) #u2
    red_proportion = 1-green_proportion 
    print(f"green list 단어 비율 : {green_proportion}")
    print(f"red list 단어 비율 : {red_proportion}")
    #expected_proportion = 0.5 #green list 를 모를 것이다 -> H0 #u1
    #std_error = np.sqrt(expected_proportion * (1 - expected_proportion) / len(input_ids))
    #z_score = 2*(green_counts - len(input_ids_list)/2)/np.sqrt(len(input_ids_list)) #기댓값에서 많이 벗어나면 기각
    z_score = (green_counts - len(input_ids_list)*gamma)/(np.sqrt(len(input_ids_list)*gamma*(1-gamma)))
    #p_value = norm.sf(np.abs(z_score)) #기댓값에 못미치면 green list를 안다!
    
    with open('Z_score_g=0.5_d=100.0.txt', 'a') as f:  # with 문을 사용하여 파일을 열면 자동으로 닫힙니다.
        f.write(str(z_score) + '\n')
            
    print(f"Z-score: {z_score}")
    if z_score > 4 :
        return True
    else :
        return False

#g=0.7, d=10.0 ->red list 단어 비율 : 0.580000000000000
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다. 이번 전시는 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 진행된다. 전시에는 책 읽는 즐거움이란 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로"
#g=0.5, d=50.0 -> red list 단어 비율 : 0.5252525252525253
#sentence = "책 읽기의 즐거움도 함께 느낄 것이다.이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여했다. 이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는"
#g=0.5, d=20.0 -> red list 단어 비율 : 0.53
#sentence = "책 읽기의 즐거움도 함께 느낄 것이다.이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여했다. 이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 "
#g=0.5, d=10.0 -> red list 단어 비율 : 0.5252525252525253
#sentence = "책 읽기의 즐거움도 함께 느낄 것이다.이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여했다. 이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는"
#g=0.5, d=5.0 -> red list 단어 비율 : 0.505050505050505
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다. 이번 전시는 '책과 나눔'이라는 주제 하에 지난달 27일 개막한 '책과 나눔'전을 통해 처음 선보였으며, 오는 9월까지 계속된다. 특히, 전시 기간 동안 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해"
#g=0.5, d=1.0 -> red list 단어 비율 : 0.6
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다.이번 전시는 ‘책 읽는 즐거움’을 주제로 한 ‘책 읽는 즐거움’ 전시를 비롯해 ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움"
#g=1.0, d=0.0 -> red list 단어 비율 : 0.5700000000000001
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다. 이번 전시는 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 진행된다. 전시에서는 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는"

#g=0.5
#sentence = "아침 일과 시작하기 전에 미리 준비해두면 좋겠습니다. 오늘은 전국이 대체로 맑겠습니다. 하지만 제주도는 구름이 많이 끼겠고요. 강원 영동 지역은 밤 한때 비가 조금 내리겠습니다. 내일 아침 기온 오늘과 비슷하겠습니다. 서울이 4도, 대구 6도로 출발하겠습니다. 낮 기온은 서울이 13도, 대구와 광주 16도로 오늘과 비슷하겠습니다. 바다의 물결은 동해 먼바다에서 최고 2.5미터까지 높게 일겠습니다. 내일 아침 기온 오늘과 비슷하겠습니다."

#사람
#sentence = "안정된 순방향 전파: 만약 우리가 오프셋 δ를 0으로 초기화하고 스케일 γ를 1로 초기화한다면, 각 출력 활성화는 단위 분산을 갖게 됩니다. 일반적인 네트워크에서는 이는 초기화 과정에서 순방향 전파 중에 분산이 안정적으로 유지됨을 보장합니다. 하지만 잔차 네트워크에서는 분산은 여전히 새로운 변동 요소가 각 레이어에서 입력에 추가되면서 증가해야 합니다. 그러나 잔차 블록마다 분산은 선형적으로 증가할 것입니다. k번째 레이어는 k의 기존"

#사람
#sentence = "이건 사람이 작성한 글입니다. 절대 절대 p-value 가 낮게 나와서도 안되고 z-score가 0 근처여서도 안됩니다. 제발 좀 잘 나와보자 제발제발 할말이 없네 이게 다야"
#detection(sentence, 25600, 0.5)

In [118]:
# Set the repetition penalty
penalty_coefficient = 1.2

# Function to apply dynamic repetition penalty to logits
def apply_repetition_penalty(logits, token_counts, penalty_coefficient):
    penalty_factors = np.ones(logits.shape[-1])

    for token_id, count in enumerate(token_counts):
        if count > 0:
            penalty_factors[token_id] = penalty_coefficient ** count

    # Apply the calculated penalties to logits
    return logits / penalty_factors

In [115]:
# def lsoftmax(x):
#     z = x-max(x)
#     return (z - np.log(np.sum(np.exp(z),keepdims=True)))

In [134]:

with open('Z_score_g=0.5_d=100.0.txt', 'w') as f:  #file내용 clear
    f.write('')
    
with open('perplexity.txt', 'w') as f:  #file내용 clear
    f.write('')
    
for line in lines:
    line = line.strip()
    print(f"prompt2 : {line}")

    input_text = line
    input_ids = tokenizer.encode(input_text, return_tensors='tf')
    initial_ids = input_ids
    #term = 3

    last_index = input_ids.shape[-1]
    nlls = []
            
    with mirrored_strategy.scope():
        for _ in tqdm(range(50)):  # 최대 100번 반복
            if input_ids.shape[-1] >= 50:
                break  # input_ids의 길이가 100 이상이면 반복 중단
            
            #print("================================")
            #print(f"문장의 토큰 개수 : {input_ids.shape[-1]}")
            #set 나눠서 green set 로짓을 촉진하기
            gamma = 0.5
            delta = 2.0
            Logits = model(input_ids).logits
            #print(f"original Logits : {Logits}")
            
            #seed 생성
            seed = input_ids[-1] % Logits.shape[-1]
            #print(f"seed : {seed[-1]}")
            np.random.seed(seed[-1])

            #random green list 설정
            indices = np.arange(Logits.shape[-1]) #0~51999
            green_list_size = int(Logits.shape[-1] * gamma)  #25600
            green_list = np.random.choice(indices, green_list_size, replace=False)
            #print(f"green list : {green_list} {green_list[200]} {green_list[1000]}") #25600개의 green list

            #--여기부터 probability 구하기
            # green logit과 red logit의 값을 담을 리스트 생성
            green_logits = tf.zeros(green_list_size)
            red_logits = tf.zeros(Logits.shape[-1] - green_list_size)

            #그린 리스트에 속한 idx 값의 Logit에는 델타 추가!!!!!!
            Logits_num = np.zeros((0,last_index-1,Logits.shape[-1]))
            Logits_num = Logits.numpy()
            Logits_num[0, -1, :] = apply_repetition_penalty(Logits_num[0, -1, :], token_counts, penalty_coefficient)
            
            for i in green_list :
                Logits_num[0,last_index-1,i] = Logits_num[0,last_index-1,i]+delta
            Logits = tf.convert_to_tensor(Logits_num, dtype=tf.float32)
            #print(Logits)
            #stable_logits = Logits - max(Logits)
            #print(stable_logits)
            #print(f"updated Logits : {Logits}")
            
            probs = tf.nn.softmax(Logits[0,last_index-1,:])
            #print(probs)
            
            #softmax 계산을 위한 분모
            #denominator = tf.reduce_sum(tf.exp(stable_logits), axis=-1)  
            #print(f"denominator : {denominator}") #25600개의 green list
            #Logits 텐서에 softmax 적용하여 확률 계산 (병렬 계산), softmax_tensor = probability
            #softmax_tensor = Logits.numpy()
            #softmax_tensor[0, last_index-1, :] = np.exp(softmax_tensor[0, last_index-1, :])
            #softmax_tensor[0, last_index-1, :] /= denominator[0,last_index-1]
            #print(softmax_tensor.shape)
            #print(f"Probabilities : {softmax_tensor}")

            # 데이터 타입 확인 및 조정
            #add_token_id = tf.argmax(softmax_tensor[0][last_index-1])
            #print(f"max : {max(softmax_tensor[0][last_index-1])}")
            #print(f"updated Logits Max value : {add_token_id}")
            
            add_token_id = tf.argmax(probs)
            #print(f"max : {max(probs)}")
            
            token_counts[add_token_id] += 1
            
            # Calculate perplexity
            # 참고 https://huggingface.co/docs/transformers/main/ko/perplexity
            #selected_probability = softmax_tensor[0, last_index-1, add_token_id]
            #nlls.append(np.log(selected_probability))
            selected_probability = probs[add_token_id]
            nlls.append(np.log(selected_probability))
            
            #print(softmax_tensor[0, last_index-1, add_token_id])

            # 텐서 연결
            add_token_id = tf.reshape(add_token_id, [1,1])
            add_token_id = tf.cast(add_token_id, input_ids.dtype)
            input_ids = tf.concat([input_ids, add_token_id], axis=-1)
            next_token = tokenizer.decode(input_ids.numpy()[0,:])
            #print(f"next token : {next_token}")
            
            last_index = input_ids.shape[-1]
            #term = 3
        final_token = tokenizer.decode(input_ids.numpy()[0,:])
        print(final_token)

        # Calculate perplexity
        perplexity = tf.exp(-tf.reduce_mean(nlls))
        print("Perplexity: ", perplexity.numpy())
        
        final_token = tokenizer.decode(input_ids.numpy()[0,:])
        print(final_token)
        print(f"is this sentence written by machine? {detection(final_token, Logits.shape[-1], gamma, initial_ids)}")

        # Calculate perplexity
        perplexity = tf.exp(-tf.reduce_mean(nlls))
        print("Perplexity: ", perplexity.numpy())
        
        perplexity_str = str(perplexity.numpy())  # perplexity를 문자열로 변환
        with open('perplexity.txt', 'a') as f:  # with 문을 사용하여 파일을 열면 자동으로 닫힙니다.
            f.write(perplexity_str + '\n')

prompt2 : 아침 일과 시작하기


 92%|█████████▏| 46/50 [00:07<00:00,  6.50it/s]


아침 일과 시작하기 전이 되면 잠잘 곳을 찾아야 한다”고도 경고한다.
잠자리는 하루 7~8시간 정도 길게 뻗었다
폈다를 반복하므로 적당한 간격을 유지하며 자는 습관을 기르는 지혜가 요구된다.
취침을 전후로 눕거나 엎드려 자는 습관도 좋지않아지
Perplexity:  7.3643174
아침 일과 시작하기 전이 되면 잠잘 곳을 찾아야 한다”고도 경고한다.
잠자리는 하루 7~8시간 정도 길게 뻗었다
폈다를 반복하므로 적당한 간격을 유지하며 자는 습관을 기르는 지혜가 요구된다.
취침을 전후로 눕거나 엎드려 자는 습관도 좋지않아지
green list 단어 개수 : 48
red list 단어 개수 : 2
green list 단어 비율 : 0.96
red list 단어 비율 : 0.040000000000000036
Z-score: 6.505382386916237
is this sentence written by machine? True
Perplexity:  7.3643174
prompt2 : 책 읽기의 즐거움


 90%|█████████ | 45/50 [00:06<00:00,  6.53it/s]


책 읽기의 즐거움 이외에 새로운 차원의 깨달음의 기쁨을 발견할 때가 많다.
좋은 글은 나쁜 글, 나쁜 글, 악한 글이 구별되게 마련이다.
좋은 글의 핵심은 바른 글이다.
바른 글에 쓰인 문장은 바른 문장 안에서 이루어진다.
바른 문장이 옳음을 증명하는 근거는 첫째, 문장의 짜임새 둘째, 문장 안의
Perplexity:  11.545321
책 읽기의 즐거움 이외에 새로운 차원의 깨달음의 기쁨을 발견할 때가 많다.
좋은 글은 나쁜 글, 나쁜 글, 악한 글이 구별되게 마련이다.
좋은 글의 핵심은 바른 글이다.
바른 글에 쓰인 문장은 바른 문장 안에서 이루어진다.
바른 문장이 옳음을 증명하는 근거는 첫째, 문장의 짜임새 둘째, 문장 안의
green list 단어 개수 : 43
red list 단어 개수 : 7
green list 단어 비율 : 0.86
red list 단어 비율 : 0.14
Z-score: 5.091168824543142
is this sentence written by machine? True
Perplexity:  11.545321
prompt2 : 가족과 함께 하는 저녁 식사


...

지역사회와의 소통 강화 및 지역사회 공헌을 촉진하기 위해 노력한 공로를 인정받았다.
쌍방울 임직원 모임인 굿윌굿윌굿네이처는 1999년 설립되어 매월 첫째 주에 모여 이웃사랑의 정을 나누고, 정기적인 봉사기간에 걸쳐 소외 계층 가정을 찾아가 명절 음식과 부식세트 배달 봉투를
Perplexity:  8.296552
지역사회와의 소통 강화 및 지역사회 공헌을 촉진하기 위해 노력한 공로를 인정받았다.
쌍방울 임직원 모임인 굿윌굿윌굿네이처는 1999년 설립되어 매월 첫째 주에 모여 이웃사랑의 정을 나누고, 정기적인 봉사기간에 걸쳐 소외 계층 가정을 찾아가 명절 음식과 부식세트 배달 봉투를
green list 단어 개수 : 47
red list 단어 개수 : 3
green list 단어 비율 : 0.94
red list 단어 비율 : 0.06000000000000005
Z-score: 6.222539674441618
is this sentence written by machine? True
Perplexity:  8.296552
prompt2 : 가족과의 소중한 대화


 92%|█████████▏| 46/50 [00:07<00:00,  6.56it/s]


가족과의 소중한 대화 기회를 갖지 못했다는 아쉬움은 컸다.
평상시엔 부모님의 간섭받지 않기 바빴는지, 평소 친밀하게 지냈는지를 묻고 싶은 마음이 굴뚝 같았다.
부모님은 딸이 학교에 다닐 때부터 줄곧 곁에 머물렀다.
초등학교 시절에는 선생님의 권유로 학급 친구들과 어울리는 친구가 생겼다.

Perplexity:  10.638528
가족과의 소중한 대화 기회를 갖지 못했다는 아쉬움은 컸다.
평상시엔 부모님의 간섭받지 않기 바빴는지, 평소 친밀하게 지냈는지를 묻고 싶은 마음이 굴뚝 같았다.
부모님은 딸이 학교에 다닐 때부터 줄곧 곁에 머물렀다.
초등학교 시절에는 선생님의 권유로 학급 친구들과 어울리는 친구가 생겼다.

green list 단어 개수 : 45
red list 단어 개수 : 5
green list 단어 비율 : 0.9
red list 단어 비율 : 0.09999999999999998
Z-score: 5.65685424949238
is this sentence written by machine? True
Perplexity:  10.638528
prompt2 : 개인 발전을 위한 목표 설정


 90%|█████████ | 45/50 [00:06<00:00,  6.54it/s]

개인 발전을 위한 목표 설정’이라는 과제를 제시했다.
곽 신임 이사장은 연세대(언론대학원 석사과정 재학 중이던 1989년 동아일보 기자로 입사하면서부터 신문사 논설위원, 편집국 부지부장직을 역임했다.
조선일보 재직시절이던 1994년 동아일보 신춘문예 등단작이 단편소설 ‘부엌데기’(동아일보 신춘호),
Perplexity:  7.239161
개인 발전을 위한 목표 설정’이라는 과제를 제시했다.
곽 신임 이사장은 연세대(언론대학원 석사과정 재학 중이던 1989년 동아일보 기자로 입사하면서부터 신문사 논설위원, 편집국 부지부장직을 역임했다.
조선일보 재직시절이던 1994년 동아일보 신춘문예 등단작이 단편소설 ‘부엌데기’(동아일보 신춘호),
green list 단어 개수 : 45
red list 단어 개수 : 5
green list 단어 비율 : 0.9
red list 단어 비율 : 0.09999999999999998
Z-score: 5.65685424949238
is this sentence written by machine? True
Perplexity:  7.239161


In [ ]:
np.random.seed(9154)
green_list_size = int(Logits.shape[-1] * gamma)  #25600
green_list = np.random.choice(indices, green_list_size, replace=False)
print(f"green list : {green_list} {green_list.shape}") #25600개의 green list

green list : [18100 37771 17147 ... 26412 26911 43793] (25600,)


In [ ]:
#model_generate

sent = '아침 일과 시작하기'
input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
with mirrored_strategy.scope():
    output = model.generate(input_ids,
                        max_length=100,
                        use_cache=False,
                        repetition_penalty=2.0)
    output_ids = output.numpy().tolist()[0]
    print(tokenizer.decode(output_ids))